In [88]:
import pandas as pd
import numpy as np

In [89]:
df=pd.read_csv('covid.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


### Counting unique categories in columns

In [90]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [91]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

### Train-Test split

In [92]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.3,random_state=0)

In [93]:
x_train

,age,gender,fever,cough,city
60,24,Female,102.0,Strong,Bangalore
80,14,Female,99.0,Mild,Mumbai
90,59,Female,99.0,Strong,Delhi
68,54,Female,104.0,Strong,Kolkata
51,11,Female,100.0,Strong,Kolkata
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [94]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### Without using Column Transformer

In [95]:
# Simple Imputer for handling missing values in fever column
from sklearn.impute import SimpleImputer
si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[['fever']])
x_test_fever=si.transform(x_test[['fever']])
x_train_fever.shape

(70, 1)

In [96]:
# Ordinal encoding -> cough
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=oe.fit_transform(x_train[['cough']])
x_test_cough=oe.transform(x_test[['cough']])

x_train_cough.shape

(70, 1)

In [97]:
# OneHotEncoder -> gender,city
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city=ohe.transform(x_test[['gender','city']])
x_train_gender_city.shape

(70, 4)

In [98]:
# Extracting Age
x_train_age=x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age=x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(70, 1)

In [99]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(70, 7)

### Using column Transformer

In [100]:
from sklearn.compose import ColumnTransformer

In [101]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [102]:
transformer.fit_transform(x_train).shape

(70, 7)

In [103]:
transformer.transform(x_test)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  25.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.    